In [1]:
# Set up CUDA
#First Change runtime to GPU and run this cell
!pip install git+https://github.com/afnan47/cuda.git
%load_ext nvcc_plugin


  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-zrwcpj00
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-zrwcpj00
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4290 sha256=40d78be52d979d31b28f349bd42ba4a4267d7c4e60cab447851b767a7c617954
  Stored in directory: /tmp/pip-ephem-wheel-cache-r3iyed5q/wheels/bc/4e/e0/2d86bd15f671dbeb32144013f1159dba09757fde36dc51a963
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [9]:
pip install pycuda


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 9.1 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1-cp311-cp311-linux_x86_64.whl size=660425 sha256=23249956253b77b3813008da093ca06dd43da9934d86272e7f1763f71e0274ec
  Stored in directory: /root/.cache/pip/wheels/77/7e/6c/d2d1451ea6424cdc3d67b36c16fa7111eafdf2034bc3405666
Successfully built pycuda


In [12]:
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
from pycuda.compiler import SourceModule

# Define CUDA kernel for matrix multiplication
kernel_code = """
__global__ void matmul(int *A, int *B, int *C, int N)
{
    int row = threadIdx.y + blockIdx.y * blockDim.y;
    int col = threadIdx.x + blockIdx.x * blockDim.x;

    if (row < N && col < N)
    {
        int sum = 0;
        for (int i = 0; i < N; i++)
        {
            sum += A[row * N + i] * B[i * N + col];
        }
        C[row * N + col] = sum;
    }
}
"""

# Initialize matrices with random values
def initialize_matrix(N):
    return np.random.randint(0, 10, (N, N)).astype(np.int32)

# Print matrix
def print_matrix(mat):
    for row in mat:
        print(" ".join(map(str, row)))

# Matrix size (NxN)
N = 4

# Initialize matrices A and B
A = initialize_matrix(N)
B = initialize_matrix(N)

# Print matrices A and B
print("Matrix A:")
print_matrix(A)
print("Matrix B:")
print_matrix(B)

# Allocate memory on the device
A_gpu = cuda.mem_alloc(A.nbytes)
B_gpu = cuda.mem_alloc(B.nbytes)
C_gpu = cuda.mem_alloc(A.nbytes)

# Copy input matrices to the device
cuda.memcpy_htod(A_gpu, A)
cuda.memcpy_htod(B_gpu, B)

# Compile the CUDA code
mod = SourceModule(kernel_code)

# Get the kernel function
matmul_kernel = mod.get_function("matmul")

# Define number of threads per block and blocks per grid
threads_per_block = (16, 16, 1)  # Block size (16x16 threads per block)
blocks_per_grid = (int(np.ceil(N / threads_per_block[0])),
                   int(np.ceil(N / threads_per_block[1])))  # Grid size

# Launch the CUDA kernel
matmul_kernel(A_gpu, B_gpu, C_gpu, np.int32(N), block=threads_per_block, grid=blocks_per_grid)

# Copy the result from device to host
C = np.empty_like(A)
cuda.memcpy_dtoh(C, C_gpu)

# Print the result
print("Matrix multiplication result (C = A * B):")
print_matrix(C)


Matrix A:
1 5 4 5
3 0 3 5
1 4 4 0
4 6 8 0
Matrix B:
3 3 2 3
2 3 6 5
5 8 4 2
6 2 8 6
Matrix multiplication result (C = A * B):
63 60 88 66
54 43 58 45
31 47 42 31
64 94 76 58
